# Eat Safe, Love

## Part 1: Set Up the Database (and Jupyter Notebook)
In terminal, import the data from establishments.json by running the following command: `mongoimport --db uk_food --collection establishments --file resources/establishments.json --jsonArray`

In [1]:
from pymongo import MongoClient
from pprint import pprint

In [2]:
mongo = MongoClient(port=27017)

In [3]:
mongo.list_database_names()

['admin', 'config', 'local', 'uk_food']

In [4]:
uk_food_db = mongo["uk_food"]

In [5]:
uk_food_db.list_collection_names()


['establishments']

In [6]:
pprint(uk_food_db["establishments"].find_one())

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('678066cfdaeae20bf64beee9'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [7]:
uk_food_establishments = uk_food_db["establishments"]

## Part 2: Update the Database

### Part 2.1: Add the new data

In [8]:
# for re-runs
uk_food_establishments.delete_many({"BusinessName":"Penang Flavours"})

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

In [9]:
new_restaurant_data = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [10]:
uk_food_establishments.insert_one(new_restaurant_data)

InsertOneResult(ObjectId('678082907914e9814ae84e2d'), acknowledged=True)

In [11]:
uk_food_establishments.find_one({"BusinessName":"Penang Flavours"})

{'_id': ObjectId('678082907914e9814ae84e2d'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

### Part 2.2: Clean the new data

In [12]:
rcc_businesstypeid = uk_food_establishments.find_one(
    {"BusinessType":"Restaurant/Cafe/Canteen"},
    {"BusinessType":1, "BusinessTypeID":1}
)

rcc_businesstypeid

{'_id': ObjectId('678066cfdaeae20bf64beee9'),
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1}

In [13]:
uk_food_establishments.update_one(
    {"BusinessName":"Penang Flavours"},
    {"$set":{"BusinessTypeID":rcc_businesstypeid["BusinessTypeID"]}}
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [14]:
uk_food_establishments.find_one({"BusinessName":"Penang Flavours"})

{'_id': ObjectId('678082907914e9814ae84e2d'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

### Part 2.3: Remove establishments with LocalAuthorityName "Dover"

### Part 2.4: Convert latitude, longitude, and RatingValue to numbers